In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Quick PP - Batch Interpretation

In [ ]:
from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import estimate_vsh_gr, calc_r35_perm
from quick_pp.core_calibration import sw_shf_leverett_j
from quick_pp.plotter import plotly_log

import pickle

# Load ROCK_FLAG
with open(r'data\04_project\MOCK_carbonate\outputs\rt_model.qppm', 'rb') as file:
    rt_model = pickle.load(file)

# Load R35 model
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
    r35_model = pickle.load(file)

In [ ]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

df = project.get_all_data()  # [['WELL_NAME', 'DEPTH', 'ZONES', 'GR', 'RT', 'NPHI', 'RHOB', 'CALI', 'BS']]
# df = all_data.copy()
for well_name, well_data in df.groupby('WELL_NAME'):
    # Load well object
    well = project.get_well(well_name)
    well_data['CPORE'] = well_data['CORE_POR'] / 100
    well_data['CPERM'] = well_data['CORE_PERM']

    # Clean up data
    well_data = badhole_flagging(well_data)
    well_data = mask_outside_threshold(well_data, fill=True)

    # Initialize lithology model
    args = {
        'litho_model': 'carb',
        'dry_calc_point': (.0, 2.71),
        'dry_clay_point': (.3, 2.7),
        'silt_line_angle': 116,
        'wet_clay_point': (0.43, 2.6),
        'sw_water_salinity': 2e5,
        'sw_m': 2.2,
        'sw_n': 2,
        'hc_corr_angle': 50,
        'hc_buffer': 0.1,
        'ressum_cutoffs': dict(
            VSHALE=.5,
            PHIT=0,
            SWT=1
        ),
    }
    vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=200)
    carb_model = Carbonate(**args)
    vclw, vcalc, vdolo = carb_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr, # pef=well_data['PEF'],
        model=model, method=method, normalize= True, carbonate_type=carbonate_type
    )
    args.update(carb_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Implement hydrocarbon correction
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,
        dry_min1_point=args['dry_calc_point'],
        dry_clay_point=args['dry_clay_point'],
        corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
    )

    # # Skip HC correction for now
    # nphihc, rhobhc = well_data['NPHI'], well_data['RHOB']

    # Estimate lithology
    carb_model = Carbonate(**args)
    vclw, vcalc, vdolo = carb_model.estimate_lithology(
        nphihc, rhobhc,  gr=well_data['GR'], vsh_gr=vsh_gr, # pef=well_data['PEF'],
        model=model, method=method, normalize= True, carbonate_type=carbonate_type
    )
    # Estimate porosity
    phit = neu_den_xplot_poro(
        nphihc, rhobhc, model='carb',
        dry_min1_point=args['dry_calc_point'],
        dry_clay_point=args['dry_clay_point'],
    )

    # PHID needs unnormalized lithology
    vclw_un, vcalc_un, vdolo_un = Carbonate(**args).estimate_lithology(
        nphihc, rhobhc, gr=well_data['GR'], vsh_gr=vsh_gr, # pef=well_data['PEF'],
        model=model, method=method, normalize= False, carbonate_type=carbonate_type
    )
    rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
    phid = density_porosity(rhobhc, rho_ma)
    
    input_features = ['GR', 'NPHI', 'RHOB', 'RT']
    # Predict rock type
    rock_flag_ml = rt_model.predict(well_data[input_features])
    
    # Predict permeability
    r35_ml = 10**(r35_model.predict(well_data[input_features]))
    perm_r35_ml = calc_r35_perm(r35_ml, phit)

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
    water_salinity = args['sw_water_salinity']
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    m = args['sw_m']
    swt = archie_saturation(well_data['RT'], rw, phit, m=m)
    
    # Estimate SHF
    shf_params = {
        1: (.007, 1.2),
        2: (.012, 1.29),
        3: (.017, 1.34),
        4: (.018, 1.13),
    }

    ift = 32
    theta = 30
    ghc = .837
    gw = 1.0

    fwl = 8380

    a = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[0])
    b = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[1])
    shf = sw_shf_leverett_j(
        perm_r35_ml, phit, well_data['DEPTH'], gw=gw, ghc=ghc,
        fwl=fwl, ift=ift, theta=theta, a=a, b=b
    )

    # Estimate reservoir summary
    well_data['ZONES'] = 'ALL'
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm_r35_ml,
                                       zones=well_data['ZONES'], cutoffs=args['ressum_cutoffs'])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhobhc
    # well_data['HC_FLAG'] = hc_flag
    well_data['VCALC'] = vcalc
    well_data['VDOLO'] = vdolo
    well_data['VCLW'] = vclw
    well_data['PHID'] = phid
    well_data['PHIT'] = phit
    well_data['SWT'] = swt
    well_data['SHF'] = shf
    well_data['PERM'] = perm_r35_ml
    well_data['VHC'] = phit * (1 - swt)
    well_data['ROCK_FLAG'] = rock_flag_ml
    well_data['ROCK_FLAG'] = well_data['ROCK_FLAG'].astype(int)

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)
    # break


In [ ]:
# Plot individual results
fig = plotly_log(well_data, well.depth_uom)
fig.show(config=dict(scrollZoom=True))

In [ ]:
from quick_pp.qaqc import quick_compare
from quick_pp.utils import zone_flagging

df = project.get_all_data()
df['VSHALE'] = df['VCLW']
df = zone_flagging(df)
compare_df, fig = quick_compare(df, return_fig=True)